In [1]:
from nltk.tokenize import sent_tokenize

import ollama

import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans, MiniBatchKMeans
from scipy.spatial.distance import cdist, pdist
from sklearn.metrics import pairwise_distances


from reco_systems.filter import filter_df
from reco_systems.llm import *
from reco_systems.CF_knn import *
from reco_systems.user_game_matrix import *
from reco_systems.evaluation import *
from reco_systems.text_filtering import *
from reco_systems.lemmatization import *
from reco_systems.embeds_utils import *

from surprise import NMF
from surprise import Dataset
from surprise.reader import Reader

sns.set_theme()
%load_ext autoreload
%autoreload 2

/Users/bsh2022/Study/venv/lib/python3.11/site-packages/treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
/Users/bsh2022/Study/venv/lib/python3.11/site-packages/treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
/Users/bsh2022/Study/venv/lib/python3.11/site-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/Users/bsh2022/Study/venv/lib/python3.11/site-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
comments_embed = pd.read_parquet("../generated_data/comments_embed.parquet")

In [ ]:
df = pd.read_parquet("special_user_comments.parquet")
user_info = rev_filter[rev_filter["User id"] == 9701][["Comment body", "Game id"]].rename(columns={"Comment body":"comment", "Game id":"game id"})
df = df[~df["game id"].isin(user_info["game id"])]
df = pd.concat([df, user_info])
df.shape

(906, 2)

In [32]:
games = pd.read_json("games_info.json", orient="records")[["game index"]]
comments = pd.read_json("special_user_comments.json", orient="records")
comments = comments.merge(games, on="game index", how="right")
comments = comments.fillna("")
comments.to_json("special_user_comments.json", orient="records")

In [ ]:
#df = df.drop(columns="game id")
#df.loc[:, "comment"] = df["comment"].str.replace("Voici le commentaire généré :\n\n", "")


In [3]:
folder = "../database_cleaned"
avis_clean  = pd.read_csv(f"{folder}/avis_clean.csv", index_col=0)
jeux_clean  = pd.read_csv(f"{folder}/jeux_clean.csv", index_col=0)
users       = pd.read_csv(f"{folder}/users.csv", index_col=0)

min_reviews = 10 
rev_filter = filter_df(avis_clean, min_reviews)

rev_filter = rev_filter.assign(index=rev_filter.index)
rev_filter["Length"] = rev_filter["Comment body"].str.split().apply(len)

# lemmas = pd.read_csv("../generated_data/Lemmas_VER_cleaned.csv")
# corpus = construction_corpus(lemmas, 5000)
# lemmas = lemmas[lemmas["Lemma"].isin(corpus)]

# comments_lemmatized = lemmas.groupby("Comment line")["Lemma"].apply(" ".join).reset_index()
# rev_filter = rev_filter[rev_filter["index"].isin(comments_lemmatized["Comment line"])]
# comments_lemmatized = comments_lemmatized.merge(rev_filter[["Game id", "User id", "index"]], left_on="Comment line", right_on="index")

# rev_filter, _ = center_score(rev_filter)
# users_means = rev_filter[["User id", "Rating"]].groupby("User id").mean().reset_index()

11201 10549
2252 2676
1957 2618
1947 2616
1946 2615
1945 2614


In [23]:
_, _, users_table, games_table = get_matrix_user_game(rev_filter)
games_ids = games_table[games_table.index.isin([503,405,689,985,1380])]
games_ids

405     1535
503     1904
689     2645
985     3895
1380    5415
Name: Game id, dtype: int64

In [24]:
import textwrap
summaries = []
for game_id in [1904]:
    comments = rev_filter[rev_filter["Game id"] == game_id]["Comment body"].reset_index()
    comments = comments.assign(Length=comments["Comment body"].str.split().apply(len))
   
    comments = comments.assign(Batch=assign_batch_number(comments, 2300))
   
    comments_batched = comments.groupby("Batch")["Comment body"].apply("\n ".join).tolist()
    # print(comments_batched)
    response = call_model_by_batch(comments_batched, "combine_phrases")
    print(textwrap.fill(response['message']['content'], width=100))
    summaries.append(response['message']['content'])

4
Nb tokens: 4566
Voici un résumé et une analyse du jeu Command & Colors : Ancients :  **Résumé**  Command & Colors :
Ancients est un jeu de stratégie à cartes qui simule les batailles antiques. Le jeu repose sur un
système de cartes qui permettent d'activer les unités sur les ailes et le centre, ou d'activer des
unités "leaders" qui vont à leur tour commander des troupes. Les joueurs doivent utiliser leurs
unités efficacement pour gagner la bataille.  **Analyse**  Le jeu offre plusieurs avantages :  * Le
système de cartes est économique, efficace et intelligent. * Les unités ont beaucoup de
caractéristiques spécifiques, ce qui les rend différenciées. * Le combat est rapide et intéressant.
* Le jeu récompense bien les meilleurs tacticiens.  Cependant, il y a quelques limites :  * Le jeu
ne prend pas en compte les tactiques de positionnement, ce qui peut rendre le combat un peu
simpliste. * Les unités combattent à l'aide de jets de dés et sans charte de combat, ce qui peut
rendre le rés

In [ ]:
Command & Colors : Ancients est un jeu de stratégie à cartes qui simule les batailles antiques. Le jeu repose sur un
système de cartes qui permettent d'activer les unités sur les ailes et le centre, ou d'activer des
unités "leaders" qui vont à leur tour commander des troupes. Les joueurs doivent utiliser leurs
unités efficacement pour gagner la bataille. Le jeu offre plusieurs avantages :  le
système de cartes est économique, efficace et intelligent. * Les unités ont beaucoup de
caractéristiques spécifiques, ce qui les rend différenciées. * Le combat est rapide et intéressant.
* Le jeu récompense bien les meilleurs tacticiens.  Cependant, il y a quelques limites :  * Le jeu
ne prend pas en compte les tactiques de positionnement, ce qui peut rendre le combat un peu
simpliste. * Les unités combattent à l'aide de jets de dés et sans charte de combat, ce qui peut
rendre le résultat aléatoire. * Les scénarios sont bien balancés, mais il y a quelques exceptions.
**Avantages**  * Le jeu est tactique, malin et rapide. * Il permet de faire des mini-campagnes en
une soirée. * La communauté de joueurs est active et enthousiaste.  **Inconvénients**  * Les blocs
de bois peuvent être difficiles à coller. * La carte peut être un peu laide. * Le système de combat
peut être aléatoire, ce qui peut rendre le jeu moins stratégique que préféré par certains joueurs.
En conclusion, Command & Colors : Ancients est un jeu de stratégie à cartes qui offre une expérience
de jeu engageante et divertissante. Bien qu'il ait quelques limites, le jeu est fidèle aux règles
des batailles antiques et offre une bonne balance entre tactique et chance.

In [ ]:
summaries = ["Le Cast News, un casse-tête magnifique ! Il te présente sous la forme d'une boîte métallique octogonale avec deux morceaux perfectionnement imbriqués, mais rien ne démontre son mécanisme. S'ensuit une frustration longue avant que tu ne trouves la solution en se fier à l'audition et le toucher. Une fois résolu, c'est magique ! Le look est incroyablement réussi, mais je l'ai trouvé un peu frustrant pour ce que cela pourrait être avec plus de manipulations possibles",
             "Positifs :\n * Le jeu permet de se plonger dans les batailles antiques avec un temps de jeu court (45 minutes à une heure)\n * La diversité des unités et des cartes tactiques permettent de choisir des combinaisons stratégiques intéressantes\n * Le système de jeu est dynamique, malgré le fait que peu d'unités se déplacent à chaque tour\n * Le commandement incertain est rendu de manière simple par les cartes\n * Les extensions offrent de nouvelles possibilités et scénarios\n Négatifs :\n * Les blocs de bois sur lesquels coller les images représentant les soldats sont long et fastidieux à mettre en place\n * La carte et les hexagones de terrain peuvent être difficiles à lire et à comprendre\n * Certaines unités se ressemblent trop, ce qui peut rendre difficile pour les joueurs de distinguer entre elles\n Critiques : Le jeu ne prévoit pas d'avantage pour attaquer un ennemi du flanc ou de derrière, ce qui peut être considéré comme un manque de réalisme * La règle permettant aux unités d'ignorer un drapeau si elles sont entourées de deux autres unités alliées peut être considérée comme un peu trop simple\n Conclusion :\n *Le jeu est considéré comme une bonne initiation au wargame, avec des règles simples et accessibles\n * Les joueurs apprécient la complexité du système de jeu, les possibilités de stratégie et les scénarios intéressants\n * Malgré quelques critiques, le jeu est considéré comme un excellent choix pour ceux qui veulent se plonger dans les batailles antiques",
             "Dr Jekyll & Mr Hyde est un jeu de plis très original et déroutant qui se joue à quatre en équipes de deux. Chaque joueur doit prendre en compte les cartes de son équipe ainsi que celles de l'équipe adverse, ce qui ajoute une dimension tactique et stratégique au jeu. Le thème du jeu est bien rendu par la mécanique du jeu, où vous devez naviguer entre la dualité personnelle de chaque joueur en jouant des cartes opposées. Il est facile à apprécier pour les fans de jeux de plis et de cartes, mais peut prendre quelques parties pour maîtriser les mécanismes. Le graphisme n'est pas très impressionnant, mais cela ne tient pas compte de l'expérience ludique exceptionnelle que ce jeu vous offre.",
             "Je suis sous le charme de Here I Stand, ce jeu de société historique qui me transporte dans l'Europe du début du 16ème siècle. Les règles peuvent être complexes, mais la richesse du thème et la diversité des possibilités sont récompensantes. Je vais avoir à choisir entre la diplomatie, la stratégie militaire ou les actions religieuses pour gagner ma partie, ce qui ajouté à l'asymétrie des différents factions, me permet de vivre une expérience unique. Malgré quelques réserves sur l'équilibrage et la diversité des options pour chaque faction, Here I Stand est un jeu qui vaut la peine d'être joué, surtout pour les amateurs de grands jeux historiques.",
             "Je suis tout à fait impressionné par cette nouvelle extension de Game of Thrones ! La carte de Westeros est un vrai coup de maître, elle offre une nouvelle dynamique de jeu qui me permet de jouer de manière très équilibrée en quatre joueurs. Les nouveaux héros sont incroyables, je les adore déjà et je suis impatient de voir ce qu'ils pourront apporter à la stratégie du jeu.  Les cartes stratégiques et les personnages ajoutent un nouveau niveau de complexité au jeu, ce qui me fait émerveiller. La météo est également une nouvelle dimension qui enrichit le jeu, avec des effets qui sont vraiment pertinents dans certaines situations.  J'ai eu quelques doutes sur l'équilibre à la début, mais après quelques parties, je me suis rendu compte que c'était juste une question de habitude. Le Lannister est effectivement un peu trop puissant, mais en revanche, les autres joueurs sont bien équilibrés.  Je ne peux pas attendre de jouer avec ces nouveaux éléments ! C'est une extension qui va rendre le jeu encore plus addictif et immersive. Je recommande vivement à tous les fans du jeu pour s'en débarrasser"]
df = pd.DataFrame(data={"summary":summaries, "game index":[405, 503, 689,985,1380]})
df.to_json("summaries.json", orient="records")

In [4]:
comments_embed = pd.read_parquet("../generated_data/comments_embed.parquet")
comments_embed = comments_embed[comments_embed["Phrases"].str.contains(r'[a-zA-Z]', regex=True)]
comments_embed = comments_embed.drop_duplicates(subset="Phrases", keep="first")

all_embeds = np.array(comments_embed["Embedding"].tolist())
kmeans = MiniBatchKMeans(n_clusters=500, batch_size=1024, random_state=42, verbose=1) 

distances = kmeans.fit_transform(all_embeds) 
comments_clusters = comments_embed.assign(Cluster=kmeans.labels_, Distance_centroid=np.take_along_axis(distances, kmeans.labels_[..., None], axis=1).flatten())

mean_dist = comments_clusters.groupby("Cluster")["Distance_centroid"].mean().reset_index()
# sns.lineplot(data=mean_dist, x="Cluster", y="Distance_centroid")

#comments_clusters = comments_clusters[comments_clusters["Cluster"].isin(mean_dist.loc[mean_dist["Distance_centroid"] < 0.7, "Cluster"])]
# #comments_clusters = comments_clusters[comments_clusters["Cluster"].isin(count.loc[count["count"] >= 10, "Cluster"])]

preserved_clusters = np.sort(comments_clusters["Cluster"].unique())
all_centroids = kmeans.cluster_centers_[preserved_clusters]

Init 1/1 with method k-means++
Inertia for init 1/1: 1874.0931439188566
[MiniBatchKMeans] Reassigning 186 cluster centers.
Minibatch step 1/57794: mean batch inertia: 0.612100804982912
Minibatch step 2/57794: mean batch inertia: 0.47141482762394804, ewa inertia: 0.47141482762394804
Minibatch step 3/57794: mean batch inertia: 0.4521543612600809, ewa inertia: 0.4713481758801346
Minibatch step 4/57794: mean batch inertia: 0.43557699514581605, ewa inertia: 0.4712243880377654
Minibatch step 5/57794: mean batch inertia: 0.42944104753127, ewa inertia: 0.4710797948355814
[MiniBatchKMeans] Reassigning 310 cluster centers.
Minibatch step 6/57794: mean batch inertia: 0.42630020123876616, ewa inertia: 0.47092483295957593
Minibatch step 7/57794: mean batch inertia: 0.42868558256668554, ewa inertia: 0.4707786620599273
Minibatch step 8/57794: mean batch inertia: 0.42484491336941993, ewa inertia: 0.4706197061762913
Minibatch step 9/57794: mean batch inertia: 0.4208420154115401, ewa inertia: 0.47044744

In [53]:
matrix_ratings, mask_ratings, users_table, games_table = get_matrix_user_game(rev_filter)
clusters_weights = cluster_weight_entropy(comments_clusters)
cos_dist_matrix = calc_similarity_matrix(matrix_ratings, mask_ratings, "cos")

users_table = users_table.to_frame().reset_index().rename(columns={"index":"User index"})
games_table = games_table.to_frame().reset_index().rename(columns={"index":"Game index"})

user_id = 208 
user_ind = users_table[users_table["User id"] == user_id]["User index"].item()
similar_users = get_KNN(cos_dist_matrix, 200, user_ind)
pred_ratings, could_predict = predict_ratings_baseline(matrix_ratings, mask_ratings, similar_users, cos_dist_matrix, user_ind)

similar_users = users_table[users_table["User index"].isin(similar_users)]["User id"].values
games_ids = games_table[games_table["Game index"].isin(could_predict)]["Game id"].values
games_ids

array([    6,    15,    25, ..., 10514, 10531, 10546])

In [54]:
def find_phrases_game(game_id, similar_users, comments_clusters, clusters_weights, weight_type="entropy"):

    other_users_comments = comments_clusters[comments_clusters["User id"].isin(similar_users) & (comments_clusters["Game id"] == game_id)]
    
    if (other_users_comments.empty):
        print(other_users_comments)

        return []

    phrases, phrases_per_cluster = 15, 1
    sim_clusters = other_users_comments["Cluster"].unique()
    n_clusters, n_clusters_goal = len(sim_clusters), phrases // phrases_per_cluster

    if n_clusters > n_clusters_goal:  # clusters selection should be made -> 1 phrase per cluster
        n_clusters = n_clusters_goal
        cw = clusters_weights[clusters_weights["Cluster"].isin(sim_clusters)].sort_values("Weight")
        if n_clusters > 1:
            half = ceil(n_clusters / 2)
            cw = pd.concat([cw.tail(half), cw.head(half)])["Cluster"].values

        else:
            if weight_type == "count":  # take with fewest phrases (in a hope to be more specific)
                cw = pd.concat([cw.head(n_clusters)])["Cluster"].values
            else:  # entropy -> take with highest entropy
                cw = pd.concat([cw.tail(n_clusters)])["Cluster"].values

        other_users_comments = other_users_comments[other_users_comments["Cluster"].isin(cw)]
    else:
        phrases_per_cluster = ceil(phrases / n_clusters)  # -> several phrases per cluster

    # In each cluster find mean embedding
    cluster_means = other_users_comments.groupby("Cluster")["Embedding"].mean().reset_index()
    phrases = []

    cw = clusters_weights.set_index("Cluster")
    for cluster, mean in cluster_means.itertuples(index=False):
        
        mean = mean.reshape(1, -1)

        cluster_embeds = comments_clusters[comments_clusters["Cluster"] == cluster]

        distances = euclidean_distances(np.array(cluster_embeds["Embedding"].tolist()), mean).flatten()
        kth = min(distances.size, phrases_per_cluster)
        dist_sorted = np.argpartition(distances, kth-1)[:kth]
        sel_phrases = cluster_embeds.iloc[dist_sorted]["Phrases"]
        print(cluster, sel_phrases)
        phrases.extend(sel_phrases)

    return phrases, other_users_comments

In [35]:
comments_embed = pd.read_parquet("../generated_data/comments_embed.parquet")
comments_embed = comments_embed[comments_embed["Phrases"].str.contains(r'[a-zA-Z]', regex=True)]

In [55]:
rated_games = rev_filter[rev_filter["User id"] == 208]["Game id"].unique()
non_rated_games = np.setdiff1d(rev_filter["Game id"].unique(), rated_games)
rated_games.shape

(1111,)

In [56]:
games_count = rev_filter[rev_filter["Game id"].isin(games_ids)].groupby("Game id")["User id"].count().reset_index()
games_count[games_count["Game id"].isin(non_rated_games)].sort_values("User id", ascending=False).head(20)

,Game id,User id
190,721,144
1041,4164,143
1186,4673,142
941,3736,140
236,908,139
1228,4882,138
1107,4414,135
1549,6184,135
409,1576,133
461,1773,133


In [40]:
comments_clusters2 = comments_clusters[comments_clusters["Cluster"].isin(mean_dist.loc[mean_dist["Distance_centroid"] < 0.7, "Cluster"])]


In [57]:
all_games_phrases, other_phrases = None, None
for i, game_id in enumerate([4164]):
    all_games_phrases, other_phrases = find_phrases_game(game_id, similar_users, comments_clusters2, clusters_weights)
    print(i)
    break
    # if game_phrases == []:
    #     print("NOTHING", game_id)
    all_games_phrases.append(game_phrases)
all_games_phrases

1 9011    ça donne des situation plutôt drôles.
Name: Phrases, dtype: object
88 134477    Le jeu n'est donc pas facile à maîtriser.
Name: Phrases, dtype: object
129 29846    J'aime les jeux avec une tension (genre ça va ...
Name: Phrases, dtype: object
131 29821    Bon, mais alors il est où le 5/5 ?
Name: Phrases, dtype: object
133 8972    Par contre, essayez de trouver des joueurs qui...
Name: Phrases, dtype: object
152 8993    a) Jouer bourré, c'est mieuxb) Jouer avec des ...
Name: Phrases, dtype: object
178 29838    Il était une fois est le genre de jeu tellemen...
Name: Phrases, dtype: object
195 9024    Certains vont adorer, d'autres detesteront.
Name: Phrases, dtype: object
208 8977    On peut y jouer à mon avis surtout avec l'inte...
Name: Phrases, dtype: object
243 29809    de jeux.
Name: Phrases, dtype: object
265 29824    Ca vaut le coup.
Name: Phrases, dtype: object
308 8966    C'est un jeu pour les imaginatifs, pour les ge...
Name: Phrases, dtype: object
319 9013    J'ai re

['ça donne des situation plutôt drôles.',
 "Le jeu n'est donc pas facile à maîtriser.",
 "J'aime les jeux avec une tension (genre ça va finir et il me manque 2 points) et des règles non discutables.",
 'Bon, mais alors il est où le 5/5 ?',
 "Par contre, essayez de trouver des joueurs qui vont se mettre d'accord sur le principe suivant : on ne va pas essayer de gagner un jeu, mais de raconter une histoire.",
 'a) Jouer bourré, c\'est mieuxb) Jouer avec des gens qui ont de l\'imagination, c\'est encore mieuxc) Jouer en changeant d\'univers (remplacez "Epée" par "Sabre laser" et, hop, on glisse dans Star Wars)',
 'Il était une fois est le genre de jeu tellement simple qu\'on "aurait pu y penser".',
 "Certains vont adorer, d'autres detesteront.",
 "On peut y jouer à mon avis surtout avec l'intention vague de gagner, mais de gagner tous en même temps en se créant collectivement une belle histoire.",
 'de jeux.',
 'Ca vaut le coup.',
 "C'est un jeu pour les imaginatifs, pour les gens qui n'o

In [64]:
other_phrases[other_phrases["Cluster"] == 88]["Phrases"].tolist()

['Un succès exagéré pour un jeu trop daté.',
 "Ce jeu n'est tout de même pas si aisé d'accès.",
 "Il était une fois fait partie de ces jeux qu'il est vraiment difficile de noter.",
 "Le jeu est lui aussi assez difficile, et arriver à faire la fin qu'on nous impose peut relever de l'exploit !",
 'Le système de jeu n’est d’ailleurs pas exempt de défauts.',
 "Pour beaucoup d'autres joueurs, ce jeu était trop déroutant."]

In [86]:
df = pd.DataFrame(data={"Phrases":all_games_phrases, "Game id":games_ids})
#df.to_parquet("fabrice_phrases.parquet")

In [ ]:
df = df.explode("Phrases")
df = df[~df["Phrases"].isna()]
df = df.assign(Length=df["Phrases"].str.split().apply(len))

In [101]:
import textwrap
comments = []
for i, game in enumerate(df["Game id"].unique()):
    print("Index", i, game)
    phrases = df[df["Game id"] == game]
    phrases = phrases.assign(Batch=assign_batch_number(phrases, 2300))
    phrases_batched = phrases.groupby("Batch")["Phrases"].apply("\n ".join).tolist()

    response = call_model_by_batch(phrases_batched, "combine_phrases")
    print(textwrap.fill(response['message']['content'], width=100))
    comments.append(response['message']['content'])

Index 0 62
1
Nb tokens: 657
Voici le commentaire généré :  C'est un jeu à deux sympa mais difficile à maitriser, intéressant
pour les amateurs, avec un système de vente et de "faire campagne" qui ajoute une touche de
stratégie. Le thème est relativement absent, mais le gameplay se passe bien malgré cela. Il y a
quelques imprécisions dans la règle, mais avec un peu d'attention, on peut tout contrôler. C'est pas
un jeu à jouer tous les soirs, mais il reste agréable et offre un bon équilibre entre complexité et
jouabilité.
Index 1 72
1
Nb tokens: 709
C'est un jeu d'une simplicité enfantine où le but est de forcer ton adversaire à dépasser 11 pommes
d'une même couleur, en accumulant des points avec des cartes objectives et en saisissant les bonnes
occasions pour récupérer des ressources ou des bâtiments.
Index 2 76
1
Nb tokens: 530
Le matériel est très mauvais, les cartes et les pions sont de mauvaise qualité, mais l'idée était
bonne. Les règles sont simples et la durée de vie du jeu est t

In [105]:
comments = pd.DataFrame(data={"comment":comments, "game id":df["Game id"].unique()})
comments.to_parquet("special_user_comments.parquet")

In [104]:
len(comments), len(games_ids)

(906, 910)

In [106]:
df["Game id"].unique()

array([   62,    72,    76,   102,   111,   112,   113,   120,   145,
         165,   225,   237,   245,   250,   267,   275,   277,   281,
         282,   305,   310,   313,   345,   350,   354,   361,   398,
         405,   408,   414,   425,   428,   458,   472,   475,   489,
         498,   513,   518,   528,   532,   538,   551,   562,   591,
         594,   614,   618,   619,   635,   638,   639,   640,   649,
         656,   690,   694,   698,   705,   712,   713,   715,   717,
         721,   723,   731,   735,   754,   765,   768,   769,   770,
         789,   793,   801,   805,   809,   818,   819,   831,   837,
         842,   893,   900,   908,   918,   920,   923,   927,   934,
         948,   952,   962,   963,   969,   978,   985,   986,   989,
        1007,  1019,  1020,  1057,  1058,  1061,  1062,  1079,  1099,
        1104,  1108,  1114,  1136,  1138,  1160,  1176,  1185,  1193,
        1205,  1219,  1250,  1257,  1261,  1271,  1282,  1294,  1302,
        1324,  1325,